In [7]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from dynamic_model.dataset import DynamicsModelDataset
from dynamic_model.model import DynamicsModel
from dynamic_model.train import train_model




In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(f"Using Device: {device}")
torch.backends.cudnn.benchmark = True

Using Device: cuda:0


## Load Dataset

In [9]:

def load_dataset(file_path, input_key, output_key, test_size= 0.3, device=device):

    dataset = np.load(file_path, allow_pickle=True)
    x = dataset[()][input_key]
    y = dataset[()][output_key]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
    train_dataset = DynamicsModelDataset(x_train,y_train, device)
    validation_dataset = DynamicsModelDataset(x_test,y_test, device)
    return train_dataset, validation_dataset


dataset_input_key = 'merged_input'
dataset_output_key = 'merged_output'
dataset_file_path = 'tmp/ib-out/ib-samples.npy'




In [ ]:

batch_size = 8000
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0,
          'drop_last':True}



train_dataset, val_dataset = load_dataset(file_path=dataset_file_path, input_key=dataset_input_key, output_key= dataset_output_key)
train_loader = DataLoader(train_dataset, **params)
val_loader = DataLoader(val_dataset, **params)

In [ ]:
num_of_features = train_dataset.get_features_size()
seq_len = train_dataset.get_seq_len()
hidden_size = 32
out_size = train_dataset.get_output_feature_size()


model = DynamicsModel(features=num_of_features,hidden_size=hidden_size,out_size=out_size, batch_size=batch_size, seq_len=seq_len, n_layers=2, dropout_p=0.7).to(device=device)

In [ ]:
train_model(model=model, train_loader=train_loader, test_loader=val_loader, n_epochs=10000, learning_rate=1e-2)